## K -means Algorithm

In [1]:
import numpy as np

In [2]:
import scipy.io as sio
juneMatrix = sio.loadmat('C:/Users/vijee/OneDrive/Desktop/Assignment/Sem-2/MLSP/Assignment2/data (1)/june.mat')
juneMatrix = juneMatrix["june"]

decemberMatrix = sio.loadmat('C:/Users/vijee/OneDrive/Desktop/Assignment/Sem-2/MLSP/Assignment2/data (1)/december.mat')
decemberMatrix = decemberMatrix["december"]

In [3]:
xDisparity =  decemberMatrix[:,0] - juneMatrix[:,0] 
len(xDisparity)

2700

In [4]:
from matplotlib import pyplot as plt
plt.hist(xDisparity)
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
# K - Means Algorithm

import random
k = 2
cluster1Mean = random.choice(xDisparity)
cluster2Mean = random.choice(xDisparity)

while cluster1Mean == cluster2Mean:
    cluster2Mean = random.choice(xDisparity)
    
prevCluster1Mean = -1
prevCluster2Mean =  -1
uMatrix =[]
cluster1Mat =[]
cluster2Mat=[]
while(True):
    tempMatrix = []
    cluster1 =[]
    cluster2=[]
    if cluster1Mean != prevCluster1Mean  and cluster2Mean != prevCluster2Mean:
        for i in range(len(xDisparity)):
            if abs(int(xDisparity[i]) - cluster1Mean) <  abs(int(xDisparity[i]) - cluster2Mean):
                cluster1.append(xDisparity[i])
                tempMatrix.append((1,0))
            else:
                cluster2.append(xDisparity[i])
                tempMatrix.append((0,1))
        prevCluster1Mean, prevCluster2Mean = cluster1Mean, cluster2Mean
        print(cluster1Mean," ",cluster2Mean)
        
        uMatrix = tempMatrix
        cluster1Mat = cluster1
        cluster2Mat = cluster2
        #print(uMatrix)
        cluster1Mean = np.mean(cluster1)
        cluster2Mean = np.mean(cluster2)
    else:
        break
uMatrix = np.array(uMatrix)
#print(cluster2Mat)
print("Mean of Cluster 1 is", cluster1Mean , "and Mean of Cluster 2 is", cluster2Mean)

38   18
40.34421364985163   20.718475073313783
40.738576449912124   22.443396226415093
40.997744700045104   23.488612836438925
41.22664199814986   24.358736059479554
Mean of Cluster 1 is 41.22664199814986 and Mean of Cluster 2 is 24.358736059479554


## EM Algorithm

In [7]:
# Initializing GMM
import math
N = len(xDisparity)
pro1 = uMatrix[:,0]

# Initial Cluster 1 Probability
count = 0
for i in range(len(pro1)):
    if pro1[i] == 1:
        count+= 1

iniProb1 = count/ len(pro1)

pro2 = uMatrix[:,1]

# Initial Cluster 2 Probability
count2 = 0
for i in range(len(pro2)):
    if pro2[i] == 1:
        count2+= 1

iniProb2 = count2/ len(pro2)

A =[]
A.append(iniProb1)
A.append(iniProb2)

iniMean = [] 
iniMean.append(cluster1Mean)
iniMean.append(cluster2Mean)


# Initial Cluster 1 Variance
iniCov1 = 0
for i in range(len(cluster1Mat)):
    iniCov1 += (cluster1Mat[i] - cluster1Mean)**2
iniCov1 = iniCov1/len(cluster1Mat)


# Initial Cluster 2 Variance
iniCov2 = 0
for i in range(len(cluster2Mat)):
    iniCov2 += (cluster2Mat[i] - cluster2Mean)**2
iniCov2 = iniCov2/len(cluster2Mat)

iniVar =[]
iniVar.append(iniCov1)
iniVar.append(iniCov2)

# Creating Membership Matrix
iniU = np.zeros((N,2))

for i in range(N):
    for j in range(2):
        f = A[j] * (1) * math.exp((-1/2 * (xDisparity[i] - iniMean[j]) * (xDisparity[i] - iniMean[j])) / (iniVar[j])) / (math.sqrt(2 * math.pi * (iniVar[j])))
        if j == 0:
            a = 1
        else:
            a = 0
        tem = A[a] *(1) * math.exp((-1/2 * (xDisparity[i] - iniMean[a]) * (xDisparity[i] - iniMean[a]))/ (iniVar[a])) / (math.sqrt(2 * math.pi * (iniVar[a])))
        iniU[i,j] = f / (f + tem)

iniU = np.array(iniU)

In [8]:
# EM Algorithm for GMM
import math
N = len(xDisparity)

emcluster1Mean = cluster1Mean
emcluster2Mean = cluster2Mean
prevCluster1Mean = -1
prevCluster2Mean = -1
clusterCov1 = 0
clusterCov2 =  0
emP1 = 0
emP2 = 0
memberMatrix = iniU


while(True):
    
    if (round(emcluster1Mean,3) != round(prevCluster1Mean,3) and round(emcluster2Mean,3) != round(prevCluster2Mean,3)):
        
        prevCluster1Mean, prevCluster2Mean = emcluster1Mean, emcluster2Mean
        
        # Cluster1 - Mean
        emMeans1 = 0
        total1 = 0
        arr1 = memberMatrix[:,0]
        for j in range(len(arr1)):
            emMeans1 += (arr1[j] * xDisparity[j])
            total1 += arr1[j]
        emMeans1 = emMeans1/ total1
        emcluster1Mean = emMeans1
        #print(emMeans1)
        #print(emMeans1.dtype)
        
       
        # Cluster2 - Mean
        emMeans2 = 0
        total2 = 0
        arr2 = memberMatrix[:,1]
        for j in range(len(arr2)):
            emMeans2 += (arr2[j] * xDisparity[j])
            total2 += arr2[j]
        emMeans2 = emMeans2/ total2
        emcluster2Mean = emMeans2
       
    
        # Cluster1 - Probability
        P1 = 0
        temp1 = memberMatrix[:,0]
        for j in range(len(temp1)):
            P1 += temp1[j]
        P1 = P1 / N
        emP1 = P1


        # Cluster2 - Probability
        P2 = 0
        temp2 = memberMatrix[:,1]
        for j in range(len(temp2)):
            P2 += temp2[j]
        P2 = P2 / N
        emP2 = P2


        # Cluster1 - Covariance
        emCov1 =  0
        res1 = memberMatrix[:,0]
        covTotal1 = 0

        for j in range(len(res1)):
            emCov1 += res1[j] * (xDisparity[j] - emMeans1) * (xDisparity[j] - emMeans1) 
            covTotal1 += res1[j]
        emCov1 =   emCov1 / covTotal1
        clusterCov1 = emCov1

        # Cluster2 - Covariance
        emCov2 =  0
        res2 = memberMatrix[:,1]
        covTotal2 = 0 

        for j in range(len(res2)):
            emCov2 += res2[j] * (xDisparity[j] - emMeans2) * (xDisparity[j] - emMeans2) 
            covTotal2 += res2[j]
        emCov2 =   emCov2 / covTotal2
        clusterCov2 = emCov2
        
        

        # creating a Membership matrix

        #pdfFunction =  math.e ** (-1 *1j * (2 * math.pi * f* n /N))
        P =[]
        P.append(P1)
        P.append(P2)
        cov= []
        cov.append(emCov1)
        cov.append(emCov2)
        Mean =[]
        Mean.append(emMeans1)
        Mean.append(emMeans2)
        
        tempU = np.zeros((N,2))

        for i in range(N):
            for j in range(2):
                f = P[j] * (1) * math.exp((-1/2 * (xDisparity[i] - Mean[j]) * (xDisparity[i] - Mean[j])) / (cov[j])) / (math.sqrt(2 * math.pi * (cov[j])))
                if j == 0:
                    a = 1
                else:
                    a = 0
                tem = P[a] *(1) * math.exp((-1/2 * (xDisparity[i] - Mean[a]) * (xDisparity[i] - Mean[a]))/ (cov[a])) / (math.sqrt(2 * math.pi * (cov[a])))
                tempU[i,j] = f / (f + tem)
                memberMatrix = tempU
    else:
        break
        
print("Mean of Cluster 1 is",emcluster1Mean," and Mean of Cluster 2 is ", emcluster2Mean)
print("Value of P1 is" , emP1, " and Value of P2 is", emP2)
print("Variance of Cluster 1 is", clusterCov1, " and Variance of Cluster 2 is", clusterCov2)

Mean of Cluster 1 is 40.16252161589064  and Mean of Cluster 2 is  20.904867143761173
Value of P1 is 0.8807245158718914  and Value of P2 is 0.11927548412809963
Variance of Cluster 1 is 34.058329241062864  and Variance of Cluster 2 is 34.87737459162765
